In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='', project_access_token='')
pc = project.project_context

# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: UNINASSAU

### Instalando bibliotecas

In [ ]:
!pip install scikit-learn --upgrade

In [ ]:
!pip install xgboost --upgrade

In [ ]:
!pip install imblearn

In [ ]:
# Em seguida iremos importar diversas bibliotecas que serão utilizadas:

# Pacote para trabalhar com JSON
import json

# Pacote para realizar requisições HTTP
import requests

# Pacote para exploração e análise de dados
import pandas as pd

# Pacote com métodos numéricos e representações matriciais
import numpy as np

# Pacote para construção de modelo baseado na técnica Gradient Boosting
import xgboost as xgb

# Pacotes do scikit-learn para pré-processamento de dados
# "SimpleImputer" é uma transformação para preencher valores faltantes em conjuntos de dados
from sklearn.impute import SimpleImputer

# Pacotes do scikit-learn para treinamento de modelos e construção de pipelines
# Método para separação de conjunto de dados em amostras de treino e teste
from sklearn.model_selection import train_test_split
# Método para criação de modelos baseados em árvores de decisão
from sklearn.tree import DecisionTreeClassifier
# Classe para a criação de uma pipeline de machine-learning
from sklearn.pipeline import Pipeline

# Pacotes do scikit-learn para avaliação de modelos
# Métodos para validação cruzada do modelo criado
from sklearn.model_selection import KFold, cross_validate

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

## Download dos conjuntos de dados em formato .csv

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/maratonadev-br/desafio-2-2020/master/Assets/Data/dataset_desafio_2.csv
df_training_dataset = pd.read_csv(r'dataset_desafio_2.csv')
df_training_dataset.tail()

Temos 15 colunas presentes no dataset fornecido, sendo dezessete delas variáveis características (dados de entrada) e um delas uma variável-alvo (que queremos que o nosso modelo seja capaz de prever). 

As variáveis características são:

    MATRICULA       - número de matrícula do estudante
    NOME            - nome completo do estudante
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    REPROVACOES_GO  - número de reprovações na disciplina de ``Gestão Operacional``
    NOTA_DE         - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    NOTA_EM         - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    NOTA_MF         - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    NOTA_GO         - média simples das notas do aluno na disciplina de ``Gestão Operacional`` (0-10)
    INGLES          - variável binária que indica se o estudante tem conhecimento em língua inglesa (0 -> sim ou 1 -> não).
    H_AULA_PRES     - horas de estudo presencial realizadas pelo estudante
    TAREFAS_ONLINE  - número de tarefas online entregues pelo estudante
    FALTAS          - número de faltas acumuladas do estudante (todas disciplinas)
    
A variável-alvo é:

    PERFIL               - uma *string* que indica uma de cinco possibilidades: 
        "EXCELENTE"      - Estudante não necessita de mentoria
        "MUITO BOM"      - Estudante não necessita de mentoria
        "HUMANAS"        - Estudante necessita de mentoria exclusivamente em matérias com conteúdo de ciências humanas
        "EXATAS"         - Estudante necessita de mentoria apenas em disciplinas com conteúdo de ciências exatas
        "DIFICULDADE"    - Estudante necessita de mentoria em duas ou mais disciplinas
        
Com um modelo capaz de classificar um estudante em uma dessas categorias, podemos automatizar parte da mentoria estudantil através de assistentes virtuais, que serão capazes de recomendar práticas de estudo e conteúdo personalizado com base nas necessidades de cada aluno.

### Explorando os dados fornecidos

Podemos continuar a exploração dos dados fornecidos com a função ``info()``:

In [ ]:
df_training_dataset.info()

É notado que existem variáveis do tipo ``float64`` (números "decimais"), variáveis do tipo ``int64`` (números inteiros) e do tipo ``object`` (nesse caso são *strings*, ou texto). 

Como a maioria dos algoritmos de aprendizado estatístico supervisionado só aceita valores numéricos como entrada, é necessário então o pré-processamento das variáveis do tipo "object" antes de usar esse dataset como entrada para o treinamento de um modelo. Também é notado que existem valores faltantes em várias colunas. Esses valores faltantes também devem ser tratados antes de serem construídos modelos com esse conjunto de dados base.

A função ``describe()`` gera várias informações sobre as variáveis numéricas que também podem ser úteis:

In [ ]:
df_training_dataset.describe()

### Visualizações

Para visualizar o dataset fornecido, podemos utilizar as bibliotecas ``matplotlib`` e ``seaborn``:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(28, 4))

sns.countplot(ax=axes[0], x='REPROVACOES_DE', data=df_training_dataset)
sns.countplot(ax=axes[1], x='REPROVACOES_EM', data=df_training_dataset)
sns.countplot(ax=axes[2], x='REPROVACOES_MF', data=df_training_dataset)
sns.countplot(ax=axes[3], x='REPROVACOES_GO', data=df_training_dataset)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(28, 4))

sns.distplot(df_training_dataset['NOTA_DE'], ax=axes[0])
sns.distplot(df_training_dataset['NOTA_EM'], ax=axes[1])
sns.distplot(df_training_dataset['NOTA_MF'], ax=axes[2])
sns.distplot(df_training_dataset['NOTA_GO'].dropna(), ax=axes[3])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(28, 4))

sns.countplot(ax=axes[0], x='INGLES', data=df_training_dataset)
sns.countplot(ax=axes[1], x='FALTAS', data=df_training_dataset)
sns.countplot(ax=axes[2], x='H_AULA_PRES', data=df_training_dataset)
sns.countplot(ax=axes[3], x='TAREFAS_ONLINE', data=df_training_dataset)

In [ ]:
fig = plt.plot()
sns.countplot(x='PERFIL', data=df_training_dataset)

## ** ATENÇÃO **

Você pode notar pela figura acima que este dataset é desbalanceado, isto é, a quantidade de amostras para cada classe que desejamos classificar é bem discrepante. O participante é livre para adicionar ou remover **LINHAS** no dataset fornecido, inclusive utilizar bibliotecas para balanceamento com ``imblearn``.

<hr>

## Pré-processamento dos dados (Maikon)

## Transformação 1 - Remover os alunos com todas as notas zeradas

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


# All sklearn Transforms must have the `transform` and `fit` methods
class DropStudents(BaseEstimator, TransformerMixin):
    def __init__(self, min_grade):
        self.min_grade = min_grade

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos uma cópia do dataframe 'X' de entrada
        data = X.copy()
        # Remove os alunos que possuem todas as notas menores ou iguais a nota miníma.
        data = data[~((data['NOTA_DE'] <= self.min_grade) & (data['NOTA_EM'] <= self.min_grade) 
                      & (data['NOTA_MF'] <= self.min_grade) & (data['NOTA_GO'] <= self.min_grade))]
        
        return data

In [ ]:
# Essa transformação recebe como parâmetro o valor da nota mínima do estudante em todas as disciplinas.
drop_students = DropStudents(min_grade=0)

# Aplicando a transformação ``DropStudents`` ao conjunto de dados base
drop_students.fit(X=df_training_dataset)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_training_dataset = pd.DataFrame.from_records(data=drop_students.transform(X=df_training_dataset),)

## Transformação 2 - Coluna Notas

Na coluna "NOTA_MF" existiam valores maiores que 10 pontos (a pontuação máxima), sendo assim, estas notas serão ajustadas para ficarem entre 0 a 10 pontos. Na coluna "NOTA_GO" existiam valores nulos. Elas serão subsituidas pela mediana.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


# All sklearn Transforms must have the `transform` and `fit` methods
class FitGradeColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos uma cópia do dataframe 'X' de entrada
        data = X.copy()
        # Para cada coluna de notas
        for c in self.columns:
            #ajusta para 10 pontos se a pontuação for maior que 10.
            data[c] = data[c].apply(lambda x: 10 if x > 10 else x)
            #ajusta para 0 pontos se a pontuação for menor que 0.
            data[c] = data[c].apply(lambda x: 0 if x < 0 else x)  
            #ajusta as notas com valores nulos para a mediana do conjunto de notas.
            s_imputer = SimpleImputer(missing_values=np.nan, strategy='median')
            #set the grade column
            s_imputer.fit(X=data[[c]])
            #apply the simple imputer transform
            data[c] = s_imputer.transform(X=data[[c]])
        
        return data

In [ ]:
# Essa transformação recebe como parâmetro uma lista com os nomes das colunas de notas.
fit_grade_columns = FitGradeColumns(columns=["NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO"])

# Aplicando a transformação ``FitGradeColumns`` ao conjunto de dados base
fit_grade_columns.fit(X=df_training_dataset)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_training_dataset = pd.DataFrame.from_records(data=fit_grade_columns.transform(X=df_training_dataset),)

## Transformação 3 - Ajustando outliers

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


# All sklearn Transforms must have the `transform` and `fit` methods
# Os outliers serão ajustados utilizando a técnica IQR Score
class HandleOutliers(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos uma cópia do dataframe 'X' de entrada
        data = X.copy()
        # Para cada coluna que se deseja remover os outliers
        for c in self.columns:            
            # Primeiro quartil - representa 25% dos dados
            Q1 = data[c].quantile(0.25)
            # Terceiro quartil - representa 75% dos dados
            Q3 = data[c].quantile(0.75)
            #Intervalo inter-quartil (Q3-Q1)
            IQR = Q3 - Q1
            # Limite máximo
            max_lim = Q3 + 1.5 * IQR
            # Limite mínimo
            min_lim = Q1 - 1.5 * IQR
            #Set the limite mínimo
            data[c] = data[c].apply(lambda x: min_lim if x < min_lim else x)
            #Set the limite máximo
            data[c] = data[c].apply(lambda x: max_lim if x > max_lim else x)
        
        return data

In [ ]:
# Essa transformação recebe como parâmetro uma lista com os nomes das colunas em que se deseja ajustar os outliers
outliers = HandleOutliers(columns=["NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO"])

# Aplicando a transformação ``HandleOutliers`` ao conjunto de dados base
outliers.fit(X=df_training_dataset)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_training_dataset = pd.DataFrame.from_records(data=outliers.transform(X=df_training_dataset),)

## Analisando os outliers depois do pré-processamento

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(28, 4))

sns.boxplot(ax=axes[0], y='NOTA_DE', data=df_training_dataset)
sns.boxplot(ax=axes[1], y='NOTA_EM', data=df_training_dataset)
sns.boxplot(ax=axes[2], y='NOTA_MF', data=df_training_dataset)
sns.boxplot(ax=axes[3], y='NOTA_GO', data=df_training_dataset)

## Transformação 4 - Remoção de algumas colunas

#### Transformação 1: excluindo colunas do dataset

Para a criação de uma transformação de dados personalizada no scikit-learn, é necessária basicamente a criação de uma classe com os métodos ``transform`` e ``fit``. No método transform será executada a lógica da nossa transformação.

Na próxima célula é apresentado o código completo de uma transformação ``DropColumns`` para a remoção de colunas de um DataFrame pandas.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin


# All sklearn Transforms must have the `transform` and `fit` methods
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        # Retornamos um novo dataframe sem as colunas indesejadas
        return data.drop(labels=self.columns, axis='columns')

Para aplicar essa transformação em um DataFrame pandas, basta instanciar um objeto *DropColumns* e chamar o método transform().

In [ ]:
# Instanciando uma transformação DropColumns
rm_columns = DropColumns(
    columns=['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM',
       'REPROVACOES_MF', 'REPROVACOES_GO', 'INGLES', 'H_AULA_PRES']  # Essa transformação recebe como parâmetro uma lista com os nomes das colunas indesejadas
)

print(rm_columns)

DropColumns(columns=['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM',
                     'REPROVACOES_MF', 'REPROVACOES_GO', 'INGLES',
                     'H_AULA_PRES'])


In [ ]:
# Visualizando as colunas do dataset original
print("Colunas do dataset original: \n")
print(df_training_dataset.columns)

Colunas do dataset original: 

Index(['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM',
       'REPROVACOES_MF', 'REPROVACOES_GO', 'NOTA_DE', 'NOTA_EM', 'NOTA_MF',
       'NOTA_GO', 'INGLES', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS',
       'PERFIL'],
      dtype='object')


In [ ]:
# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=df_training_dataset)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_training_dataset = pd.DataFrame.from_records(
    data=rm_columns.transform(
        X=df_training_dataset
    ),
)

In [ ]:
# Visualizando as colunas do dataset transformado
print("Colunas do dataset após a transformação ``DropColumns``: \n")
print(df_training_dataset.columns)

Colunas do dataset após a transformação ``DropColumns``: 

Index(['NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'TAREFAS_ONLINE', 'FALTAS',
       'PERFIL'],
      dtype='object')


#### Transformação 2: tratando dados faltantes

Para tratar os dados faltantes em nosso conjunto de dados, iremos agora utilizar uma transformação pronta da biblioteca scikit-learn, chamada **SimpleImputer**.

Essa transformação permite diversas estratégias para o tratamento de dados faltantes. A documentação oficial pode ser encontrada em: https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html

Neste exemplo iremos simplesmente transformar todos os valores faltantes em zero.

In [ ]:
# Criação de um objeto ``SimpleImputer``
si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=0,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

In [ ]:
# Visualizando os dados faltantes do dataset após a primeira transformação (df_data_2)
print("Valores nulos antes da transformação SimpleImputer: \n\n{}\n".format(df_training_dataset_2.isnull().sum(axis = 0)))

In [ ]:
# Aplicamos o SimpleImputer ``si`` ao conjunto de dados df_data_2 (resultado da primeira transformação)
si.fit(X=df_training_dataset_2)

# Reconstrução de um novo DataFrame Pandas com o conjunto imputado (df_data_3)
df_training_dataset_3 = pd.DataFrame.from_records(
    data=si.transform(
        X=df_training_dataset_2
    ),  # o resultado SimpleImputer.transform(<<pandas dataframe>>) é lista de listas
    columns=df_training_dataset_2.columns  # as colunas originais devem ser conservadas nessa transformação
)

In [ ]:
# Visualizando os dados faltantes do dataset após a segunda transformação (SimpleImputer) (df_data_3)
print("Valores nulos no dataset após a transformação SimpleImputer: \n\n{}\n".format(df_training_dataset_3.isnull().sum(axis = 0)))

Nota-se que não temos mais nenhum valor faltante no nosso conjunto de dados :)

Vale salientar que nem sempre a alteração dos valores faltantes por 0 é a melhor estratégia. O participante é incentivado a estudar e implementar estratégias diferentes de tratamento dos valores faltantes para aprimorar seu modelo e melhorar sua pontuação final.

### Treinando um modelo de classificação

Finalizado o pré-processamento, já temos o conjunto de dados no formato necessário para o treinamento do nosso modelo:

In [ ]:
df_training_dataset_3.head()

No exemplo fornecido, iremos utilizar todas as colunas, exceto a coluna **LABELS** como *features* (variáveis de entrada).

A variável **LABELS** será a variável-alvo do modelo, conforme descrito no enunciado do desafio.

#### Definindo as features do modelo

In [ ]:
# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_training_dataset[features]
y = df_training_dataset[target]

O conjunto de entrada (X):

In [ ]:
X.head()

,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,TAREFAS_ONLINE,FALTAS
0,6.2,5.8,4.6,5.9,4,3
1,6.0,6.2,5.2,4.5,4,3
2,7.3,6.7,7.1,7.2,0,3
3,0.0,0.0,0.0,0.0,4,4
4,0.0,0.0,0.0,0.0,2,5


As variáveis-alvo correspondentes (y):

In [ ]:
y.head()

,PERFIL
0,EXATAS
1,EXATAS
2,HUMANAS
3,DIFICULDADE
4,DIFICULDADE


## Balanceando as classes com oversampling - SMOTE

In [ ]:
smt = SMOTE()

X, y = smt.fit_resample(X, y)

#### Separando o dataset em um conjunto de treino e um conjunto de teste

Iremos separar o dataset fornecido em dois grupos: um para treinar nosso modelo, e outro para testarmos o resultado através de um teste cego. A separação do dataset pode ser feita facilmente com o método *train_test_split()* do scikit-learn:

In [ ]:
# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

<hr>

#### Criando um modelo baseado em árvores de decisão

No exemplo fornecido iremos criar um classificador baseado em **árvores de decisão**.

Material teórico sobre árvores de decisão na documentação oficial do scikit-learn: https://scikit-learn.org/stable/modules/tree.html

O primeiro passo é basicamente instanciar um objeto *DecisionTreeClassifier()* da biblioteca scikit-learn.

## Treinando o modelo com o classificador baseado em árvore de decisão

In [ ]:
# Criação de uma árvore de decisão com a biblioteca ``scikit-learn``:
decision_tree = DecisionTreeClassifier(max_depth=8)

# Treino do modelo (é chamado o método *fit()* com os conjuntos de treino)
decision_tree.fit(X_train, y_train)

# Realização de teste cego no modelo criado
y_pred = decision_tree.predict(X_test)

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))

print (classification_report(y_test, y_pred))

## Treinando o modelo c/ o algoritmo KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

# Realização de teste cego no modelo criado
y_pred = knn.predict(X_test)

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))

print (classification_report(y_test, y_pred))

## Criando um modelo XGBoost

In [ ]:
from xgboost import XGBClassifier

xg_boost = XGBClassifier()
xg_boost.fit(X_train, y_train)

# Realização de teste cego no modelo criado
y_pred = xg_boost.predict(X_test)

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))

print (classification_report(y_test, y_pred))

<hr>

## Scoring dos dados necessários para entregar a solução

Como entrega da sua solução, esperamos os resultados classificados no seguinte dataset chamado "to_be_scored.csv":

### Download da "folha de respostas"

In [ ]:
!wget --no-check-certificate --content-disposition https://raw.githubusercontent.com/vanderlei-test/dataset-uninassau/master/to_be_scored_uninassau.csv
df_to_be_scored = pd.read_csv(r'to_be_scored_uninassau.csv')
df_to_be_scored.tail()

--2020-09-06 00:11:09--  https://raw.githubusercontent.com/vanderlei-test/dataset-uninassau/master/to_be_scored_uninassau.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32172 (31K) [text/plain]
Saving to: ‘to_be_scored_uninassau.csv.1’

100%[======================================>] 32,172      --.-K/s   in 0.001s  

2020-09-06 00:11:09 (26.4 MB/s) - ‘to_be_scored_uninassau.csv.1’ saved [32172/32172]



,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
495,940576,Samuel Bahia Cranulunan,0,0,0,0,6.3,5.3,7.0,5.6,1,4,0,8
496,682956,Samuel de Linhares,0,0,0,0,5.7,5.9,5.8,5.0,1,1,1,6
497,850896,Gigi Olga de Oliveira,0,0,0,0,6.7,7.1,5.7,6.3,0,16,6,3
498,525876,Marta Oaman,0,0,0,0,6.7,7.2,5.2,5.7,1,10,3,5
499,867666,Eliel Jardel da Costa Sanches,0,0,0,0,7.1,6.8,6.5,6.0,1,5,3,6


# Atenção!

O dataframe ``to_be_scored`` é a sua "folha de respostas". Note que a coluna "PERFIL" não existe nessa amostra, que não pode ser então utilizada para treino de modelos de aprendizado supervisionado.

In [ ]:
df_to_be_scored.info()

<hr>

# Atenção!

# Para poder aplicar seu modelo e classificar a folha de respostas, você precisa primeiro aplicar as mesmas transformações com colunas que você aplicou no dataset de treino.

# Não remova ou adicione linhas na folha de respostas. 

# Não altere a ordem das linhas na folha de respostas.

# Ao final, as 500 entradas devem estar classificadas, com os valores previstos em uma coluna chamada "target"

<hr>

Na célula abaixo, repetimos rapidamente os mesmos passos de pré-processamento usados no exemplo dado com árvore de decisão

In [ ]:
# 1 - Não posso aplicar a Transformação 1, pois remove linhas.

# 2 - Ajusta as colunas notas (missing values e valor entre 0 e 10)
fit_grade_columns = FitGradeColumns(columns=["NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO"])
# Aplicando a transformação ``FitGradeColumns`` ao conjunto de dados base
fit_grade_columns.fit(X=df_to_be_scored)
# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_to_be_scored_2 = pd.DataFrame.from_records(data=fit_grade_columns.transform(X=df_to_be_scored),)

# 3 - Não apliquei o ajuste dos outliers das colunas notas

# 4 - Remoção de algumas colunas
rm_columns = DropColumns(columns=['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM',
       'REPROVACOES_MF', 'REPROVACOES_GO', 'INGLES', 'H_AULA_PRES'])
# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=df_to_be_scored_2)
# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_to_be_scored_3 = pd.DataFrame.from_records(data=rm_columns.transform(X=df_to_be_scored_2),)

df_to_be_scored_3.tail()

<hr>

Pode ser verificado abaixo que as colunas da folha de resposta agora são idênticas às que foram usadas para treinar o modelo:

In [ ]:
df_to_be_scored_3.columns

### Executando as predições na "folha de respostas"

In [ ]:
y_pred = decision_tree.predict(df_to_be_scored_3)
df_to_be_scored_3['target'] = y_pred
df_to_be_scored_3.tail()

,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,TAREFAS_ONLINE,FALTAS,target
495,6.3,5.3,7.0,5.6,0,8,HUMANAS
496,5.7,5.9,5.8,5.0,1,6,EXATAS
497,6.7,7.1,5.7,6.3,6,3,EXATAS
498,6.7,7.2,5.2,5.7,3,5,EXATAS
499,7.1,6.8,6.5,6.0,3,6,EXATAS


### Salvando a folha de respostas como um arquivo .csv para ser submetido

In [ ]:
project.save_data(file_name="results.csv", data=df_to_be_scored_3.to_csv(index=False))

{'file_name': 'results.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'uninassauchallengev20-donotdelete-pr-lkhosvqvhddqzg',
 'asset_id': '7e33ae78-5076-4fdd-903a-0ab87da6138e'}

# Atenção

# A execução da célula acima irá criar um novo "data asset" no seu projeto no Watson Studio. Você precisará realizar o download deste arquivo juntamente com este notebook e criar um arquivo zip com os arquivos **results.csv** e **notebook.ipynb** para submissão. (os arquivos devem estar nomeados desta forma)

<hr>

## Parabéns!

Se você já está satisfeito com a sua solução, vá até a página abaixo e envie os arquivos necessários para submissão.

# https://uninassau.maratona.dev
